## Python Module and Packages Lecture and Lab

This interpreter essentionally reproduces lis.py from Peter Norvig. Go read his posts.

### Environment Interface

In [2]:
import abc
class Environment(abc.ABC):
    """
    This is the interface for an Environment. The client for 
    this interface is a language intepreter. 
    """
    @classmethod
    @abc.abstractmethod
    def empty(cls):
        return cls()
    
    @abc.abstractmethod
    def extend(self, variable, value):
        """
        extend an existing environment by binding variable to value.
        The values must be an acceptable value in the language. If the
        same variable is used twice the newer value must be bound.
        """
    
    @abc.abstractmethod
    def extend_many(self, envdict):
        """
        extend the current environment by values in the dictionary
        envdict. If the dictionary contains variables already in the
        environment, the newer values from the dictionary are bound
        """
        
    @abc.abstractmethod
    def lookup(variable):
        """
        return the unique binding of the variable and the environment it was bound
        in as a tuple. If it is not found raise a NameError as below
        """
        raise NameError("{} not found in Environment".format(vaiable))

### Environment Implementation

In [3]:
class Env:
    """
    The keys k must be strings, and the values v must be legitimate values
    in our environment.
    
    The empty dictionary represents the empty environment.
    """
    
    def __init__(self, outerenv=None):
        self.env = dict()
        self.outerenv = outerenv
    
    @classmethod
    def empty(cls):
        return cls()
    
    def extend(self, variable, value):
        self.env[variable] = value
        
    def extend_many(self, envdict):
        self.env.update(envdict)

    #your code here
    def lookup(self, key):
        try:
            found = self.env[key]
            env = self
        except KeyError:
            if self.outerenv is not None:
                found, env =self.outerenv.lookup(key)
            else:
                raise NameError("{} <<>> not found in Environment".format(key))
        return found, env
            
Environment.register(Env)

__main__.Env

### Parser

In [4]:
Symbol = str

def typer(token):
    if token == 'true':
        return True
    elif token == 'false':
        return False
    try:
        t = int(token)
        return t
    except ValueError:
        try:
            t = float(token)
            return t
        except ValueError:
            return Symbol(token)
        
def lex(loc):
    tokenlist =  loc.replace('(', ' ( ').replace(')', ' ) ').split()
    return [typer(t) for t in tokenlist]

def syn(tokens):
    if len(tokens) == 0:
        return []
    token = tokens.pop(0)
    if token == '(':
        L = []
        while tokens[0] != ')':
            L.append(syn(tokens))
        tokens.pop(0) # pop off ')'
        return L
    else:
        if token==')':
            assert 1, "should not have got here"
        return token
    
def parse(loc):
    return syn(lex(loc))

### Evaluator

In [5]:
def global_env(envclass):
    "An environment with some Scheme standard procedures."
    import math, operator as op
    env = envclass.empty()
    env.extend_many(vars(math))
    env.extend_many({
        '+':op.add, '-':op.sub, '*':op.mul, '/':op.truediv, 
        'abs':     abs,
        'max':     max,
        'min':     min,
        'round':   round,
        '>':op.gt, '<':op.lt, '>=':op.ge, '<=':op.le, '==':op.eq,
        'not':     op.not_
    })
    return env

In [6]:
class Function():
    def __init__(self, params, parsedbody, env):
        self.params = params
        self.code = parsedbody
        self.env = env
        self.envclass = env.__class__
        
    def __call__(self, *args):
        funcenv = self.envclass(outerenv = self.env)
        funcenv.extend_many(zip(self.params, args))
        return eval_ptree(self.code, funcenv)

In [7]:
def eval_ptree(x, env):
    fmap={'#t':True, '#f':False, 'nil':None}
    if x in ('#t', '#f', 'nil'):
        return fmap[x]        
    elif isinstance(x, Symbol):
        # variable lookup
        return env.lookup(x)[0]
    elif not isinstance(x, list):  # constant
        return x
    elif len(x)==0: #noop
        return None
    elif x[0]=='if':
        (_, predicate, truexpr, falseexpr) = x
        if eval_ptree(predicate, env):
            expression = truexpr
        else:
            expression = falseexpr
        return eval_ptree(expression, env)
    elif x[0] == 'def':         # variable definition
        (_, var, expression) = x
        #postorder traversal by nested eval is needed below
        # your code here
        env.extend(var, eval_ptree(expression, env))
    elif x[0] == 'store':           # (set! var exp)
        (_, var, exp) = x
        env.lookup(var)[1].extend(var, eval_ptree(exp, env))
    elif x[0] == 'func':
        (_, parameters, parsedbody) = x
        return Function(parameters, parsedbody, env)
    else:                          # operator
        op = eval_ptree(x[0], env)
        #postorder traversal to get subexpressione before running the op
        args = [eval_ptree(arg, env) for arg in x[1:]]
        return op(*args)

In [8]:
class Program():
    
    def __init__(self, program, env):
        self.program = [e.strip() for e in program.split('\n')]
        self.env = env
        
    def __iter__(self):
        for line in self.program:
            yield line
    
    def parse(self):
        parses=[]
        for l in iter(self):
            parses.append(parse(l))
        return parses
            
    def run(self):
        results=[]
        for l in iter(self):
            parsed_line = parse(l)
            results.append(eval_ptree(parsed_line, self.env))
        return results

### Driver to run the code

In [9]:
def backtolang(exp):
    boolmap={True:'#t', False:'#f'}
    if  isinstance(exp, list):
        return '(' + ' '.join(map(backtolang, exp)) + ')' 
    elif isinstance(exp, bool):
        return boolmap[exp]
    elif exp is None:
        return 'nil'
    else:
        return str(exp)
    
def repl(env, prompt='calc> '):
    while True:
        try:
            val = eval_ptree(parse(input(prompt)), env)
        except (KeyboardInterrupt, EOFError):
            break
        if val is not None: 
            print(backtolang(val))
            


In [10]:
def run_program_asif_repl(program, env):
    prog=Program(prpgram, globenv)
    for result in p1c.run():
        print(backtolang(result))

In [11]:
def run_program(program, env):
    prog=Program(prpgram, globenv)
    endit = None
    for result in p1c.run():
        endit = result
    return endit

In [12]:
globenv = global_env(Env)

In [13]:
repl(globenv)# to get out of the repl in the notebook just cause an exception like below

calc> (def rad 1)
calc> a


NameError: a <<>> not found in Environment

### Tests

In [14]:
p1 = """
(def ra 5)
ra
(if (== (> 2 3) #t) #f ra)
"""

In [15]:
program = """
(def rad 5)
rad
(def radiusfunc (func (radius) (* pi (* radius radius))))
(radiusfunc rad)
(def myvar 0)
(if (== myvar 1) (store rad 6) (store rad 7))
(radiusfunc rad)
(== 1 1)
"""

In [16]:
p=Program(program, globenv)
list(iter(p))

['',
 '(def rad 5)',
 'rad',
 '(def radiusfunc (func (radius) (* pi (* radius radius))))',
 '(radiusfunc rad)',
 '(def myvar 0)',
 '(if (== myvar 1) (store rad 6) (store rad 7))',
 '(radiusfunc rad)',
 '(== 1 1)',
 '']

In [17]:
for s in p.parse():
    print(s)

[]
['def', 'rad', 5]
rad
['def', 'radiusfunc', ['func', ['radius'], ['*', 'pi', ['*', 'radius', 'radius']]]]
['radiusfunc', 'rad']
['def', 'myvar', 0]
['if', ['==', 'myvar', 1], ['store', 'rad', 6], ['store', 'rad', 7]]
['radiusfunc', 'rad']
['==', 1, 1]
[]


In [18]:
for result in p.run():
    print(backtolang(result))

nil
nil
5
nil
78.53981633974483
nil
nil
153.93804002589985
#t
nil


### pyscaffold

Pyscaffold is a project which creates a scaffolding for us for a python project. We'll use it to understand the structure of a python project and the modules that go therein.

Get into your `py35` virtual environment.
We'll see:

- the notion of a package module (also see https://docs.python.org/3/tutorial/modules.html)
- generating documentation
- doing tests on the fly
- creating command line executables.
- installing in development mode (`pip install -e .`)
- uninstalling(`pip uninstall packagename`)


In [19]:
!putup -h

usage: putup [-h] [-p NAME] [-d TEXT] [-u URL] [-l LICENSE] [-f] [-U]
             [--with-namespace NS1[.NS2]]
             [--with-cookiecutter TEMPLATE | --with-django] [--with-travis]
             [--with-pre-commit] [--with-tox] [-v]
             PROJECT

PyScaffold is a tool for easily putting up the scaffold of a Python project.

positional arguments:
  PROJECT               project name

optional arguments:
  -h, --help            show this help message and exit
  -p NAME, --package NAME
                        package name (default: project name)
  -d TEXT, --description TEXT
                        package description (default: '')
  -u URL, --url URL     package url (default: '')
  -l LICENSE, --license LICENSE
                        package license from dict_keys(['lgpl3', 'simple-bsd',
                        'mozilla', 'apache', 'none', 'gpl3', 'lgpl2',
                        'affero', 'artistic', 'proprietary', 'new-bsd', 'isc',
                        'mit', 'gpl2', '

We'll now go look inside.

`python setup.py` incantations:


- python setup.py --help-commands
- python setup.py install
- pip install -e .
- python setup.py sdist
- python setup.py tests
- python setup.py doctest
- python setup.py docs
- python setup.py --help
- python setup.py build
- python setup.py clean
- python setup.py develop

### What we will learn



Also see http://python-packaging-user-guide.readthedocs.org/en/latest/distributing/#requirements-for-packaging-and-distributing.

What we'll learn?

- how to do command lines
- how relative imports work
- installing into virtual environments (`conda create py35-test`). Also see http://conda.pydata.org/docs/using/envs.html and http://docs.python-guide.org/en/latest/dev/virtualenvs/.
- capturing environments(`pip freeze > requirements.txt`, `pip install -r requirements.txt` ) and (`conda env export > environment.yml`, `conda env create -f environment.yml`)

### play around time

```
rm -rf /tmp/fibby
pushd /tmp
putup -l mit --with-travis fibby
popd
```

## `stupidlang` as a package

Lets create it first...

Create a "skeleton project" with the incantation (mit license for example)

```
rm -rf /tmp/stupidlang
pushd /tmp
putup -l mit --with-travis stupidlang
popd
```

And lets put some files in!

In [22]:
%%bash
rm -rf /tmp/stupidlang
pushd /tmp
putup -l mit --with-travis stupidlang
popd

/tmp ~/Projects/private/cs207/FALL/labs
~/Projects/private/cs207/FALL/labs


In [23]:
%%bash
ls -l /tmp/stupidlang

total 64
-rw-r--r--  1 rahul  wheel    69 Sep 29 11:04 AUTHORS.rst
-rw-r--r--  1 rahul  wheel   128 Sep 29 11:04 CHANGES.rst
-rw-r--r--  1 rahul  wheel  1077 Sep 29 11:04 LICENSE.txt
-rw-r--r--  1 rahul  wheel   296 Sep 29 11:04 README.rst
drwxr-xr-x  9 rahul  wheel   306 Sep 29 11:04 docs
-rw-r--r--  1 rahul  wheel    57 Sep 29 11:04 requirements.txt
-rw-r--r--  1 rahul  wheel  1946 Sep 29 11:04 setup.cfg
-rw-r--r--  1 rahul  wheel   602 Sep 29 11:04 setup.py
drwxr-xr-x  4 rahul  wheel   136 Sep 29 11:04 stupidlang
-rw-r--r--  1 rahul  wheel   241 Sep 29 11:04 test-requirements.txt
drwxr-xr-x  5 rahul  wheel   170 Sep 29 11:04 tests


In [26]:
%%bash
ls -l /tmp/stupidlang/stupidlang

total 32
-rw-r--r--  1 rahul  wheel   130 Sep 29 11:04 __init__.py
-rw-r--r--  1 rahul  wheel  1150 Sep 29 11:06 env_dictimpl.py
-rw-r--r--  1 rahul  wheel  1143 Sep 29 11:05 envinterface.py
-rw-r--r--  1 rahul  wheel  2362 Sep 29 11:04 skeleton.py


In [27]:
!cat -n /tmp/stupidlang/stupidlang/__init__.py

     1	import pkg_resources
     2	
     3	try:
     4	    __version__ = pkg_resources.get_distribution(__name__).version
     5	except:
     6	    __version__ = 'unknown'


In [24]:
%%file /tmp/stupidlang/stupidlang/envinterface.py
import abc
class Environment(abc.ABC):
    """
    This is the interface for an Environment. The client for 
    this interface is a language intepreter. 
    """
    @classmethod
    @abc.abstractmethod
    def empty(cls):
        return cls()
    
    @abc.abstractmethod
    def extend(self, variable, value):
        """
        extend an existing environment by binding variable to value.
        The values must be an acceptable value in the language. If the
        same variable is used twice the newer value must be bound.
        """
    
    @abc.abstractmethod
    def extend_many(self, envdict):
        """
        extend the current environment by values in the dictionary
        envdict. If the dictionary contains variables already in the
        environment, the newer values from the dictionary are bound
        """
        
    @abc.abstractmethod
    def lookup(variable):
        """
        return the unique binding of the variable and the environment it was bound
        in as a tuple. If it is not found raise a NameError as below
        """
        raise NameError("{} not found in Environment".format(variable))

Writing /tmp/stupidlang/stupidlang/envinterface.py


In [25]:
%%file /tmp/stupidlang/stupidlang/env_dictimpl.py
from .envinterface import Environment

class Env:
    """
    Absfun: the dicionary {k1:v1, k2:v2,...} represents the
    environment binding k1 to v1 and k2 to v2. There are no duplicates.
    The keys k must be strings, and the values v must be legitimate values
    in our environment.
    The empty dictionary represents the empty environment.
    Repinv: Newer bindings replace older bindings in the dictionary.
    This is guaranteed by using python dictionaries.
    """

    def __init__(self, outerenv=None):
        self.env = dict()
        self.outerenv = outerenv

    @classmethod
    def empty(cls):
        return cls()

    def extend(self, variable, value):
        self.env[variable] = value

    def extend_many(self, envdict):
        self.env.update(envdict)

    def lookup(self, key):
        try:
            found = self.env[key]
            env = self
        except KeyError:
            if self.outerenv is not None:
                found, env =self.outerenv.lookup(key)
            else:
                raise NameError("{} <<>> not found in Environment".format(key))
        return found, env

Environment.register(Env)

Writing /tmp/stupidlang/stupidlang/env_dictimpl.py


In [28]:
%%file /tmp/stupidlang/stupidlang/parser.py
Symbol = str

def typer(token):
    if token == 'true':
        return True
    elif token == 'false':
        return False
    try:
        t = int(token)
        return t
    except ValueError:
        try:
            t = float(token)
            return t
        except ValueError:
            return Symbol(token)
        
def lex(loc):
    tokenlist =  loc.replace('(', ' ( ').replace(')', ' ) ').split()
    return [typer(t) for t in tokenlist]

def syn(tokens):
    if len(tokens) == 0:
        return []
    token = tokens.pop(0)
    if token == '(':
        L = []
        while tokens[0] != ')':
            L.append(syn(tokens))
        tokens.pop(0) # pop off ')'
        return L
    else:
        if token==')':
            assert 1, "should not have got here"
        return token
    
def parse(loc):
    return syn(lex(loc))

Writing /tmp/stupidlang/stupidlang/parser.py


In [30]:
%%file /tmp/stupidlang/stupidlang/evaluator.py
import math
import operator as op
from .parser import parse, Symbol

def global_env(envclass):
    "An environment with some Scheme standard procedures."
    env = envclass.empty()
    env.extend_many(vars(math))
    env.extend_many({
        '+':op.add, '-':op.sub, '*':op.mul, '/':op.truediv,
        'abs':     abs,
        'max':     max,
        'min':     min,
        'round':   round,
        '>':op.gt, '<':op.lt, '>=':op.ge, '<=':op.le, '==':op.eq,
        'not':     op.not_
    })
    return env

class Function():
    def __init__(self, params, parsedbody, env):
        self.params = params
        self.code = parsedbody
        self.env = env
        self.envclass = env.__class__

    def __call__(self, *args):
        funcenv = self.envclass(outerenv = self.env)
        funcenv.extend_many(zip(self.params, args))
        return eval_ptree(self.code, funcenv)

def eval_ptree(x, env):
    fmap={'#t':True, '#f':False, 'nil':None}
    if x in ('#t', '#f', 'nil'):
        return fmap[x]
    elif isinstance(x, Symbol):
        # variable lookup
        return env.lookup(x)[0]
    elif not isinstance(x, list):  # constant
        return x
    elif len(x)==0: #noop
        return None
    elif x[0]=='if':
        (_, predicate, truexpr, falseexpr) = x
        if eval_ptree(predicate, env):
            expression = truexpr
        else:
            expression = falseexpr
        return eval_ptree(expression, env)
    elif x[0] == 'def':         # variable definition
        (_, var, expression) = x
        #postorder traversal by nested eval is needed below
        # your code here
        env.extend(var, eval_ptree(expression, env))
    elif x[0] == 'store':           # (set! var exp)
        (_, var, exp) = x
        env.lookup(var)[1].extend(var, eval_ptree(exp, env))
    elif x[0] == 'func':
        (_, parameters, parsedbody) = x
        return Function(parameters, parsedbody, env)
    else:                          # operator
        op = eval_ptree(x[0], env)
        #postorder traversal to get subexpressione before running the op
        args = [eval_ptree(arg, env) for arg in x[1:]]
        return op(*args)

class Program():
    """
    The representation of the program, and a mechanism for running it.
    Methods
    -------
    __init__
        Constructor takes an io stream and an env
    __iter__
        Yields the source code of the program line by line
    parse
        Yields the program list by list, with each list being the
        parse of a line
    run
        Yields the result of running each line.
    """

    def __init__(self, program, env):
        self.program = program
        self.env = env

    def __iter__(self):
        for line in self.program:
            yield line

    def parse(self):
        for l in iter(self):
            yield parse(l)

    def run(self):
        """
        a generator that runs the program, line by line
        Yields
        ------
        str, int, float, or None
            The result of running a single line of stupidlang code.
            The lines in the program are run from beginning to end.
        """
        for l in iter(self):
            yield eval_ptree(parse(l), self.env)

Writing /tmp/stupidlang/stupidlang/evaluator.py


In [29]:
%%file /tmp/stupidlang/stupidlang/run.py
from .parser import parse
from .evaluator import eval_ptree, Program

def backtolang(exp):
    """
    Takes a expression list and converts it back into a
    stupidlang expression.
    Parameters
    ----------
    exp : list
        A list representing a parsed stupidlang expression
    Returns
    -------
    str
        A string with the corrsponding stupidlang code
    Examples
    --------
    >>> backtolang(None)
    nil
    >>> backtolang(True)
    #t
    >>> backtolang()
    """
    boolmap={True:'#t', False:'#f'}
    if  isinstance(exp, list):
        return '(' + ' '.join(map(backtolang, exp)) + ')'
    elif isinstance(exp, bool):
        return boolmap[exp]
    elif exp is None:
        return 'nil'
    else:
        return str(exp)

def repl(env, prompt='SL> '):
    """
    A REPL for the stupidlang language
    Parameters
    ----------
    env : Environment
        a concrete implementation instance of the Environment interface
    prompt : str, optional
        a string for the prompt, default SL>
    """
    try:
        import readline
    except:
        pass
    while True:
        try:
            val = eval_ptree(parse(input(prompt)), env)
        except (KeyboardInterrupt, EOFError):
            break
        if val is not None:
            print(backtolang(val))

def run_program_asif_repl(program, env):
    """
    Runs code with output as-if we were in a repl
    Parameters
    ----------
    program: str
        a multi-line string representing the stupidlang program
    env : Environment
        a concrete implementation instance of the Environment interface
    Returns
    -------
    str:
        The output of the program as if it were being run in a REPL
    """
    prog=Program(prpgram, env)
    for result in prog.run():
        print(backtolang(result))

def run_program(program, env):
    """
    Runs code without output until the last line where output is provided.
    Parameters
    ----------
    program: str
        a multi-line string representing the stupidlang program
    env : Environment
        a concrete implementation instance of the Environment interface
    Returns
    -------
    str:
        The last output of the program as if it were being run in a REPL
    """

    prog=Program(program, env)
    endit = None
    for result in prog.run():
        endit = result
    return backtolang(endit)

Writing /tmp/stupidlang/stupidlang/run.py


Now, in the file, skeleton.py, add this stuff from the definition of `fib` onwards, replacing what was there:

```python
import os
from .run import repl, run_program
from .env_dictimpl import Env
from .evaluator import global_env

def parse_args(args):
    """
    Parse command line parameters
    :param args: command line parameters as list of strings
    :return: command line parameters as :obj:`airgparse.Namespace`
    """
    parser = argparse.ArgumentParser(
        description="A stupid lispish language")
    parser.add_argument(
        '-v',
        '--version',
        action='version',
        version='stupidlang {ver}'.format(ver='0.0.0'))
    parser.add_argument(
        '-t',
        '--talkative',
        dest="loglevel",
        help="set loglevel to INFO",
        action='store_const',
        const=logging.INFO)
    parser.add_argument(
        '-tt',
        '--very-talkative',
        dest="loglevel",
        help="set loglevel to DEBUG",
        action='store_const',
        const=logging.DEBUG)
    parser.add_argument(
        '-i',
        '--interactive',
        action = 'store_true',
        help='starts a REPL to run stupidlang code')
    parser.add_argument(
        '-l',
        '--load',
    nargs=1,
        help="a file to load before running the repl, implies -i")
    parser.add_argument(
        dest="programfile",
    nargs='?',
        help="the program to run. the last value will be printed")
    ns = parser.parse_args(args)
    if (bool(ns.interactive) | bool(ns.load)) & bool(ns.programfile):
        parser.error('-i or -l cannot be given together with a program file')
    if len(args)==0:
        parser.error('atleast one of -i, -l file, or file must be specified')
    if bool(ns.load) and not os.path.isfile(ns.load[0]):
        parser.error("Loaded file must exist or be file")
    if bool(ns.programfile) and not os.path.isfile(ns.programfile):
        parser.error("Program file must exist or be file")
    return ns


def main(args):
    args = parse_args(args)
    logging.basicConfig(level=args.loglevel, stream=sys.stdout)
    _logger.debug("Starting lispy calculator...")
    env=global_env(Env)
    if args.interactive:
        repl(env)
        return None
    elif bool(args.load):
        with open(args.load[0]) as f:
            run_program(f, env)
            repl(env)
            return None
    else:
        with open(args.programfile) as f:
            output = run_program(f, env)
            _logger.info("Script ends here")
            return output



def run():
    print("stupidlang version {}".format(__version__))
    print(main(sys.argv[1:]))


if __name__ == "__main__":
    run()
```

In [32]:
%%file /tmp/stupidlang/tests/test1.sl
(def ra 5)
ra
(if (== (> 2 3) #t) #f ra)

Writing /tmp/stupidlang/tests/test1.sl


In [33]:
%%file /tmp/stupidlang/tests/test2.sl
(def rad 5)
rad
(def radiusfunc (func (radius) (* pi (* radius radius))))
(radiusfunc rad)
(def myvar 0)
(if (== myvar 1) (store rad 6) (store rad 7))
(radiusfunc rad)
(== 1 1)
(def area (radiusfunc rad))
rad

Writing /tmp/stupidlang/tests/test2.sl


And in test_skeleton.py, add this in the place of the test for `fib` there:

```python
import pytest
from stupidlang.skeleton import main

__author__ = "Rahul Dave"
__copyright__ = "Rahul Dave"
__license__ = "mit"


def test_main():
    assert main(["tests/test2.sl"])=='7'
```

When you create tests, you want to create one corresponding test file per source file.

 Our tests, following `py.test`'s recommendations, use absolute imports.

(Some projects, like pyastro, recommend relative imports for testing)

At some point, because of the absolute imports, you will need to issue an `pip install -e .` or `python setup.py develop` in the top level of your package to get things to work. This happens typically when you have multiple soource folders. I just mention it here for completeness, and to note that for tests to work, you need to be in this top level folder.

Now create a new repository `stupidlang` for yourself on github.

Commit your work, then:

`git remote add origin git@github.com:rahuldave/stupidlang.git`

`git push --set-upstream origin master`

and now you are on github.

Its your job to get things hooked up to travis. See the `travis-install.sh`, `.travis.yml` and edit appropriately.